## Prep

In [ ]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
import warnings
import seaborn as sns


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
dat_v1 = dat[dat['visit'] == 1]
dat_v2 = dat.loc[dat['visit'] == 2, ['subjid','y']].rename(columns={'y': 'y2'})
dat_v3 = dat.loc[dat['visit'] == 3, ['subjid','y']].rename(columns={'y': 'y3'})

merged_df = pd.merge(dat_v1, dat_v2, on='subjid')
merged_df = pd.merge(merged_df, dat_v3, on='subjid')

merged_df['y_tot'] = (merged_df['y'] | merged_df['y2'] | merged_df['y3']).astype(int)

dat_plt = merged_df[['y_tot', 'nbSESpc2score', 'N_UNFAV_CT00', 'nbK3paFacilities', 'G_bla_rk', 'nutrition3cat', 'PA3cat']]
dat_plt = dat_plt.rename(columns = {'y_tot': 'Y', 
                                    'nbSESpc2score': 'nb SES',
                                    'N_UNFAV_CT00': 'nb Unfavorable Food Store', 
                                    'nbK3paFacilities': 'nb Physical Activity Facilities', 
                                    'G_bla_rk': 'nb Racial Segregation (Black)', 
                                    'nutrition3cat': 'ind Nutrition Cate', 
                                    'PA3cat': 'ind Physical Activity Cate'})


## Plot

In [ ]:
correlation_matrix = dat_plt.corr()

plt.figure(figsize=(8, 6))  

sns.heatmap(correlation_matrix, cmap="Greens", square=True, xticklabels=True)

plt.title('Correlation Matrix')
plt.show()